## Домашнее задание №4 (курс "Практикум по программированию на языке Python")

### Тема: Web-сервер для Telegram-бота.

#### Преподаватели: Мурат Апишев (mel-lain@yandex.ru) и Роман Ищенко (roman.ischenko@gmail.com)

**Выдана**:   29 апреля 2021

**Дедлайн**:   21:00 13 мая 2021

**Среда выполнения**: Jupyter Notebook (Python 3.7)

#### Правила:

Результаты выполнения задания:

- архив со скриптами и файлами Dockerfile, Pipfile и Pipfile.lock, который 1-2 команды позволяет развернуть сервер, решающий поставленные в задании задачи
- Jupyter Notebook, где __весь код__ из скриптов дублируется (1 ячейка - 1 скрипт) с комментарием, содержащим информацию о том, из какого файла взят код и что верхнеуровнево этот код делает
- ссылка на чат-бота в Telegram, который автоматически начинает обработку пользовательских запросов после разворачивания сервера

__Максимальное число баллов за задание - 25__.

Работа делится на базовую и дополнительную части, __полное выполнение базовой является обязательным__! Без этого вся работа автоматически оцениваниется 0 баллов. Дополнительную часть можно решать фрагментарно, оценка будет выставляться пропорционально.

Готовое задание отправляется на почту преподавателя.

Задание выполняется самостоятельно. Если какие-то студенты будут уличены в списывании, все они автоматически получат за эту работу 0 баллов. Если вы нашли в Интернете какой-то специфичный код, который собираетесь заимствовать, обязательно укажите это в задании - наверняка вы не единственный, кто найдёт и использует эту информацию.

Удалять фрагменты формулировок заданий запрещается.

#### Постановка задачи:

- В данной работе нужно написать веб-сервер для обслуживания чат-бота в мессенджере Telegram
- Чат-бот предназначен для обработки изображений, передаваемых пользователем
- Сервер должен быть реализован на Flask + waitress
- Проект разворачивается с помощью библиотеки управления виртуальными окружениями Pipenv и технологии контейнеризации Docker

[Пример полностью решенного задания](https://t.me/mipt_python_server_bot)

### Базовая часть (12 баллов)

- Требуется реализовать чат-бота для мессенджера Telegram, который возвращает пользователю на любое входное текстовое сообщение это сообщение. Предполагается, что никакие другие данные (картинки, стикеры) бот получать не будет
- Бэкенд проекта должен быть реализован на Flask + waitress
- Для прокидывания сервера наружу требуется использовать сервис и утилиту ngrok
- Проект должен разворачиваться с помощью Docker, внутри которого создается виртуальное окружение с помощью Pipenv, решение должно разворачиваться командами:

```
docker build -t tg-bot .
docker run -d tg-bot
```

- __Обязательно (!!!)__ нужно вставить копии содержимого всех файлов проекта в этот ноутбук (1 файл - 1 ячейка) и подписать, к какому файлу относится какая ячейка и что код в ней в общих чертах делает

__Рекомендации:__

1. [Ссылка](https://core.telegram.org/bots) на инструкцию по созданию ботов в Telegram

2. Для работы с Telegram лучше всего использовать библиотеку python-telegram-bot

3. Рекомендуемая сигнатура для обработчика POST-запросов от бота:

In [ ]:
# app = Flask(__name__)

@app.route(f'/{BOT_TOKEN}', methods=['POST'])

4. Рекомендуемая структура проекта (плоская):
    - Dockerfile
    - Pipfile
    - Pipfile.lock
    - settings.py (для хранения токена, локального адреса сервера \[localhost + port\] и любых иных общих статических данных)
    - server.py (для старта локального сервера получения/отправки запросов)
    - configure_and_run.py (для настройки и старта всего сервиса)

    Опционально можно завести отдельные файлы для логики обработчика запросов, хранения текстов сообщений и любых иных целей


5. Dockerfile может иметь следующую структуру:

```
FROM python:3.7.9-stretch
# установка нужных утилит в систему, например, curl и unzip
# установка pipenv
WORKDIR /app
# копирование всех важных файлов в рабочую директорию
# запуск pipenv
ENTRYPOINT ["pipenv", "run", "python", "configure_and_run.py", "&"]
```

Последняя команда будет запускаться при вызове `docker run`

6. Вывод информации для отладки и логгирования в stdout можно делать с помощью `os.system(f'echo {message}')`

7. Поскольку локально поднятый сервер не виден снаружи, нужно как-то прокинуть его в Интернет, это можно сделать с помощью облачного хостинга или сервиса ngrok, который можно использовать через одноименную консольную утилиту, что и требуется в задании

8. Выкачивание и запуск ngrok можно выполнить в configure_and_run.py, в этом помогут утилиты curl и unzip. Для этих и других задач, требующих параллельного вызова сторонних приложений и скриптов, полезен модуль subprocess

9. После того, как ngrok будет запущен на указанном локальном адресе, нужно будет передать внешний адрес боту (выставить webhook). Достать этот адрес можно, запустив следующую команду и распарсив выходной файл (там будет json):


In [ ]:
os.system(f'curl http://localhost:4040/api/tunnels > {filename}')

10. После этого выставить webhook можно в server.py после создания и перед запуском Flask-сервера командой

In [ ]:
# bot = telegram.Bot(BOT_TOKEN)

bot.setWebhook('{URL}/{HOOK}'.format(URL=EXTERNAL_URL, HOOK=BOT_TOKEN))

11. Обработку запросов стоит оборачивать в try-except-блок и отправлять пользователю валидное сообщение об ошибке или просьбу повторить какое-то действие, саму ошибку при этому надо логгировать

12. Все вспомогательные файлы, возникающие в процессе работы, должны гарантированно удаляться, как только надобность в них отпадает

13. Запуск сервера имеет смысл производить из configure_and_run.py с помощью subprocess (полезен будет метод communicate)

In [ ]:
# src/server.py
# запуск сервера, инициализация глобальных объектов, обработка post запросов от сервера телеграмма, которые он шлет
# через прокинутый вебхук

import logging
import subprocess
import requests
import time
import json

from flask import Flask, request
from waitress import serve

from telegram import Update

from bot_entity import Telegram_bot
from config import TOKEN, LOCALHOST_URL

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

my_bot = Telegram_bot(TOKEN)
app = Flask(__name__)

@app.route('/')
def index():
    return 'ok, 200'

@app.route(f'/{TOKEN}', methods=['POST'])
def respond():
    content = Update.de_json(request.get_json(force=True), my_bot.updater.bot)
    disp = my_bot.updater.dispatcher
    disp.process_update(content)
    return 'ok, 200'

def get_ngrok_url_init():
    res_url = None
    ngrok_pid = subprocess.Popen(['./ngrok', 'http', '5000'], stdout=None)
    time.sleep(5)
    tunnel_url = requests.get(LOCALHOST_URL).text
    j = json.loads(tunnel_url)
    for url_init in j['tunnels']:
        if url_init['proto'] == 'https':
            res_url = url_init['public_url']
    return res_url

if __name__ == '__main__':
    logger.log(level=logging.INFO, msg='Initialising server')
    URL = get_ngrok_url_init()
    logger.log(level=logging.INFO, msg='Ngrok Server started')
    my_bot.throw_webhook(f'{URL}/{TOKEN}')
    logger.log(level=logging.INFO, msg='webhook established')
    serve(app, host='0.0.0.0', port=5000)

In [ ]:
# src/bot_entity.py
# инкапсуляция логики бота с обработкой сообщений и проч.

from telegram.ext import Updater, MessageHandler, CommandHandler, Filters

class Telegram_bot:
    def __init__(self, token):
        self.updater = Updater(token=token)
        msg_filt = Filters.text & (~Filters.command)
        msg_handler = MessageHandler(filters=msg_filt, callback=self.handle_message)
        err_handler = MessageHandler(filters=(~msg_filt) & (~Filters.command), callback=self.error_message)
        cmd_handler = CommandHandler(command='start', callback=self.handle_bot_start)
        self.updater.dispatcher.add_handler(msg_handler)
        self.updater.dispatcher.add_handler(err_handler)
        self.updater.dispatcher.add_handler(cmd_handler)

    def throw_webhook(self, url):
        self.updater.bot.setWebhook(url=url)

    def handle_message(self, update, context):
        chat_id = update.message.chat.id
        if update.message.text:
            text = update.message.text.encode('utf-8').decode()
            update.message.reply_text(text=text)

    def handle_bot_start(self, update, context):
        chat_id = update.message.chat.id
        update.message.reply_text('Hello, I will taunt you echoing your messages and pictures')

    def error_message(self, update, context):
        chat_id = update.message.chat.id
        update.message.reply_text('Unsupported message type')

In [ ]:
# src/config.py
# Константы: Токен бота у ботоотца телеграмма и хост

TOKEN = 'здесь длинный токен-секрет бота'
LOCALHOST_URL = "http://localhost:4040/api/tunnels"

In [ ]:
# dockerfile
# python-образ c установкой pipenv и ngrok в директорию приложения
# копирование
# запуск

FROM python:3.8.2

RUN pip install pipenv

ENV PROJECT_DIR /usr/local/bin/src/app
WORKDIR ${PROJECT_DIR}

COPY Pipfile Pipfile.lock ${PROJECT_DIR}/
RUN pipenv install --system --deploy
COPY ./src .

RUN curl https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -o ngrok-stable-linux-amd64.zip
RUN unzip ngrok-stable-linux-amd64.zip

CMD ["python", "server.py"]

## В дальнейших клетках будет только код файла src/_bot_entity.py_, т.к. при выполнении заданий ниже изменения вносились лишь в логике бота

### Дополнительная часть

__Задание 1 (2 балла):__ Доработать базовое решение до бота, который предлагает пользователю отправить изображение в формате JPG или PNG (в виде аттача), реагирует адекватно на попытки послать ему что-то иное, и в случае получения валидного изображения возвращает его в чат ответным сообщением

__Рекомендации:__

1. Нужно аккуратно обрабатывать крайние случаи (пришел файл, не являющийся изображением, или имеющий не тот формат)

2. Полезна функция, которая определяет, что сообщение содержит файл или фото:

In [ ]:
# content = telegram.Update...

@staticmethod
def is_file(content):
    return (content.message is not None and
            ((content.message.document is not None and content.message.document.file_id is not None) or
             (content.message.photo is not None and len(content.message.photo) and content.message.photo[0].file_id is not None)))

In [ ]:
# src/bot_entity.py
# добавлен фильтр на файлы-изображения и ответ на них

from telegram.ext import Updater, MessageHandler, CommandHandler, Filters

class Telegram_bot:
    def __init__(self, token):
        self.updater = Updater(token=token)
        msg_filt = Filters.document.file_extension("jpg") | Filters.document.file_extension("png") \
                   | Filters.photo | Filters.caption \
                   | Filters.text & (~Filters.command)
        msg_handler = MessageHandler(filters=msg_filt, callback=self.handle_message)
        err_handler = MessageHandler(filters=(~msg_filt) & (~Filters.command), callback=self.error_message)
        cmd_handler = CommandHandler(command='start', callback=self.handle_bot_start)
        self.updater.dispatcher.add_handler(msg_handler)
        self.updater.dispatcher.add_handler(err_handler)
        self.updater.dispatcher.add_handler(cmd_handler)

    def throw_webhook(self, url):
        self.updater.bot.setWebhook(url=url)

    def handle_message(self, update, context):
        chat_id = update.message.chat.id
        if update.message.text:
            text = update.message.text.encode('utf-8').decode()
            update.message.reply_text(text=text)
        else:
            if update.message.photo:
                update.message.reply_photo(photo=update.message.photo[0].file_id, caption=update.message.caption)
            else:
                update.message.reply_document(document=update.message.document.file_id, caption=update.message.caption)

    def handle_bot_start(self, update, context):
        chat_id = update.message.chat.id
        update.message.reply_text('Hello, I will taunt you echoing your messages and pictures')

    def error_message(self, update, context):
        chat_id = update.message.chat.id
        update.message.reply_text('Unsupported message type')

__Задание 2 (5 баллов):__ Модернизировать решение из предыдущего пункта - при получении изображения от пользователя нужно вернуть не его, а текстовое сообщение с кнопкой, в котором написано, что при нажатии кнопки полученное ранее изображение будет отправлено пользователю обратно. Это и должно происходить при нажатии на кнопку. При этом пользователь может отправить в ответ еще одно изображение, и тогда он снова получит сообщение с кнопкой, но сохранено будет уже новое изображение.

__Рекомендации:__

1. В отличие от предыдущих пунктов, здесь сессия пользователя начинает содержать состояния: пользователь еще не отправил картинку, и пользователь уже сделал это. Это означает, что стоит завести профиль пользователя, который будет идентифицироваться по `chat_id` (который используется при отправке сообщений) и содержать текущие состояние, картинку или любые иные полезные и вспомогательные данные пользователя

2. Хранить нужно не один общий профиль, а по профилю на ользователя, иначе сервис нельзя будет использоваться нескольким пользователям. Обычно профили хранятся в базах, для простоты реализации в задании достаточно ограничиться хранением в `dict` в оперативной памяти, который заводится при старте сервера

3. Нужно следить за корректностью изменения состояния сессии и профиля пользователя, в том числе и при возникновении различных ошибок

4. Полезна функция, которая определяет, что сообщение отправлено нажатием кнопки:

In [ ]:
@staticmethod
def is_button(content):
    return content.callback_query is not None and content.callback_query.data is not None

In [ ]:
# src/bot_entity.py
# добавлена логика меню-клавиатуры и словарь 
# {ключ: чат_id, {тип-файла: фото/файл, значение: id последнего отправленного изображения} }

from telegram import InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, MessageHandler, CommandHandler, Filters, CallbackQueryHandler

class Telegram_bot:
    def __init__(self, token):
        self.updater = Updater(token=token)
        msg_filt = Filters.document.file_extension('jpg') | Filters.document.file_extension('png') \
                   | Filters.photo | Filters.caption \
                   | Filters.text & (~Filters.command)

        self.updater.dispatcher.add_handler(MessageHandler(filters=msg_filt, callback=self.handle_message))
        self.updater.dispatcher.add_handler(MessageHandler(filters=(~msg_filt) & (~Filters.command), callback=self.error_message))
        self.updater.dispatcher.add_handler(CommandHandler(command='start', callback=self.handle_bot_start))
        self.updater.dispatcher.add_handler(CallbackQueryHandler(callback=self.push_button))

        self.chat_dict = {}

    def throw_webhook(self, url):
        self.updater.bot.setWebhook(url=url)

    def handle_message(self, update, context):
        chat_id = update.message.chat.id
        if update.message.text:
            text = update.message.text.encode('utf-8').decode()
            update.message.reply_text(text=text)
        else:
            button_list = [
                [
                    InlineKeyboardButton("send last picture", callback_data='1')
                ]
            ]
            reply_markup = InlineKeyboardMarkup(button_list)
            update.message.reply_text('Time to choose...', reply_markup=reply_markup)
            if update.message.photo:
                self.chat_dict[chat_id] = {'type': 'photo', 'id': update.message.photo[0].file_id }
            else:
                self.chat_dict[chat_id] = {'type': 'document', 'id': update.message.document.file_id}

    def handle_bot_start(self, update, context):
        chat_id = update.message.chat.id
        self.chat_dict[chat_id] = None
        update.message.reply_text('Hello, I will taunt you echoing your messages and pictures')

    def push_button(self, update, context):
        query = update.callback_query
        chat_id = query.message.chat.id
        query.answer()
        try:
            reply = self.chat_dict[chat_id]
            if reply['type'] == 'photo':
                query.message.reply_photo(photo=reply['id'])
            else:
                query.message.reply_document(document=reply['id'])
        except KeyError:
            query.message.reply_text(text='No photo in button')

    def error_message(self, update, context):
        update.message.reply_text('Unsupported message type')

__Задание 3 (1 балл):__ Заменить кнопку возвращения изображения на две кнопки "Размытие" и "Оттенки серого", каждая из которых применяет соответствующее преобразование к сохраненному изображению и возвращает его пользователю. Загрузка нового изображения повторяет логику из предыдущего пункта, кнопки можно нажимать неограниченное количество раз

__Рекомендации:__

1. Указанные фильтры называются "gaussian smoothing" и "grayscale"

In [ ]:
# src/bot_entity.py
# изменена логика меню-клавиатуры
# С помощью Pillow реализовано применение фильтр-эффектов к изображению

import os
import requests

from telegram import InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, MessageHandler, CommandHandler, Filters, CallbackQueryHandler
from PIL import Image, ImageFilter

class Telegram_bot:
    def __init__(self, token):
        self.updater = Updater(token=token)
        msg_filt = Filters.document.file_extension('jpg') | Filters.document.file_extension('png') \
                   | Filters.photo | Filters.caption \
                   | Filters.text & (~Filters.command)

        self.updater.dispatcher.add_handler(MessageHandler(filters=msg_filt, callback=self.handle_message))
        self.updater.dispatcher.add_handler(MessageHandler(filters=(~msg_filt) & (~Filters.command), callback=self.error_message))
        self.updater.dispatcher.add_handler(CommandHandler(command='start', callback=self.handle_bot_start))
        self.updater.dispatcher.add_handler(CallbackQueryHandler(callback=self.push_button))

        self.chat_dict = {}

    def throw_webhook(self, url):
        self.updater.bot.setWebhook(url=url)

    def handle_message(self, update, context):
        chat_id = update.message.chat.id
        if update.message.text:
            text = update.message.text.encode('utf-8').decode()
            update.message.reply_text(text=text)
        else:
            button_list = [
                [
                    InlineKeyboardButton("Blur", callback_data='blur'),
                    InlineKeyboardButton("Grayscale", callback_data='gray')
                ]
            ]
            reply_markup = InlineKeyboardMarkup(button_list)
            update.message.reply_text('Time to choose...', reply_markup=reply_markup)
            if update.message.photo:
                self.chat_dict[chat_id] = {'type': 'photo', 'id': update.message.photo[0].file_id }
            else:
                self.chat_dict[chat_id] = {'type': 'document', 'id': update.message.document.file_id}

    def handle_bot_start(self, update, context):
        chat_id = update.message.chat.id
        self.chat_dict[chat_id] = None
        update.message.reply_text('Hello, I will taunt you echoing your messages and pictures')

    def push_button(self, update, context):
        query = update.callback_query
        chat_id = query.message.chat.id
        query.answer()
        try:
            reply_pic = self.chat_dict[chat_id]
            pic_info = self.updater.bot.get_file(file_id=reply_pic['id'])
            img = Image.open(fp=requests.get(pic_info.file_path, stream=True).raw)#
            if query.data == 'blur':
                new_img = img.convert('RGB').filter(ImageFilter.GaussianBlur(radius=4))
            else:
                new_img = img.convert('LA')
            new_img.save('img.png')
            with open('img.png', 'rb') as f:
                query.message.reply_photo(photo=f)
            os.remove('img.png')
        except KeyError:
            query.message.reply_text(text='No photo in button')

    def error_message(self, update, context):
        update.message.reply_text('Unsupported message type')

__Бонусное задание (до 5 баллов):__ Предложить и реализовать функциональные, эстетические или технические улучшения реализованного бота (допускается только в случае решения всех предыдущих пунктов задания)